In [2]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, timezone

In [19]:
# Define US Geological Survey base URL
usgs_base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'

# We will retrieve a list of earthquakes one year at a time.
# Create lists of start and end dates so we can loop through them
start_year = 1995
end_year = 2024
# end_year = 1997
start_dates = [f"{year}-01-01" for year in range(start_year, end_year)]
end_dates = [f"{year}-12-31" for year in range(start_year, end_year)]
# start_dates = ["2017-01-01"]
# end_dates = ["2017-06-30"]

# print(start_dates)
# print(end_dates)

In [4]:
# Define minimum and maximum latitude and longitude of the contiguous US
min_latitude = 25.8400
max_latitude = 49.3800
min_longitude = -124.670
max_longitude = -66.9500

In [5]:
# Define minimum magnitude
min_magnitude = 3.0

In [20]:
# Construct URL with appropriate filters

# First, location filter consisting of a box around the contiguous U.S.
location_filter = f"&minlatitude={min_latitude}&minlongitude={min_longitude}&maxlatitude={max_latitude}&maxlongitude={max_longitude}"

# Second, magnitude. We only want earthquakes of magnitude greater equal 3.0
magnitude_filter = f"&minmagnitude={min_magnitude}"

# Third, dates. We retrieve data year by year from 1995 through 2023
# Create a list for the detail URLs
detail_urls = []

# Loop through years
for i in range(len(start_dates)):
    date_filter = f"&starttime={start_dates[i]}&endtime={end_dates[i]}"
    # print(start_dates[i])

    # Retrieve summary data
    summary_url = f"{usgs_base_url}{date_filter}{location_filter}{magnitude_filter}"
    # print(summary_url)
    try:
        summary_response = requests.get(summary_url).json()
        # print(json.dumps(summary_response, indent=4))
    except:
        print(f'Error: {summary_response}')
        print(f'Summary URL: {summary_url}')

    # Extract URLs that contain detailed earthquake data. That URL is contained in
    # 'features'-->'properties'-->'detail'
    # We only want the data from the 'moment-tensor' and the 'focal-mechanism' section' in the details.
    # Therefore, we only want the URL for earthquake records that contain that section. We can find
    # a list of the sections included in the detailed data in 'features'-->'properties'-->'types'.

    for feature in summary_response['features']:
        types = str(feature['properties']['types'])
        if ',moment-tensor,' in types and ',focal-mechanism,' in types:
            detail_urls.append(feature['properties']['detail'])

# print(detail_urls)
print(len(detail_urls))

1558


In [18]:
# Extract earthquake detailed data
# Define a lsit that will contain the data
earthquake_data_list = []

# Loop through each URL in the 'detail_urls' list and retrieve details about each earthquake
for url in detail_urls:
    # Retrieve the earthquake details into an earthquake dictionary.
    try:
        # print(url)
        detail_response = requests.get(url).json()
        # print(json.dumps(detail_response, indent=4))

        properties = detail_response['properties']
        geometry = detail_response['geometry']
        mt_properties = properties['products']['moment-tensor'][0]['properties']
        fm_properties = properties['products']['focal-mechanism'][0]['properties']

        mt_keys_list = ['n-axis-azimuth',
                       'n-axis-length',
                       'n-axis-plunge',
                       'p-axis-azimuth',
                       'p-axis-length',
                       'p-axis-plunge',
                       't-axis-azimuth',
                       't-axis-length',
                       't-axis-plunge',
                       'percent-double-couple',
                       'scalar-moment',
                       'tensor-mpp',
                       'tensor-mrp',
                       'tensor-mrr',
                       'tensor-mtp',
                       'tensor-mrt',
                       'tensor-mtt']
        
        fm_keys_list = ['nodal-plane-1-dip',
                        'nodal-plane-1-rake',
                        'nodal-plane-1-strike',
                        'nodal-plane-2-dip',
                        'nodal-plane-2-rake',
                        'nodal-plane-2-strike']
        
        # Make sure we only retrieve complete records
        if all(mt_key in list(mt_properties.keys()) for mt_key in mt_keys_list) and \
           all(fm_key in list(fm_properties.keys()) for fm_key in fm_keys_list):
            # Collect the data into a dictionary
            print(f"ID:{detail_response['id']}, Time: {datetime.fromtimestamp(properties['time']/1000, timezone.utc)}")
            dict_earthquake = {'id': detail_response['id'],
                              'time': datetime.fromtimestamp(properties['time']/1000, timezone.utc),
                              'place': properties['place'],
                              'longitude': geometry['coordinates'][0],
                              'latitude': geometry['coordinates'][1],
                              'depth': geometry['coordinates'][2],
                              'magnitude': properties['mag'],
                              'felt': properties['felt'],
                              'cdi': properties['cdi'],
                              'mmi': properties['mmi'],
                              'significance': properties['sig'],
                              'number_stations': properties['nst'],
                              'min_staion_distance': properties['dmin'],
                              'nodal_plane_1_dip': fm_properties['nodal-plane-1-dip'],
                              'nodal_plane_1_rake': fm_properties['nodal-plane-1-rake'],
                              'nodal_plane_1_strike': fm_properties['nodal-plane-1-strike'],
                              'nodal_plane_2_dip': fm_properties['nodal-plane-2-dip'],
                              'nodal_plane_2_rake': fm_properties['nodal-plane-2-rake'],
                              'nodal_plane_2_strike': fm_properties['nodal-plane-2-strike'],
                              'n_axis_azimuth': mt_properties['n-axis-azimuth'],
                              'n_axis_length' : mt_properties['n-axis-length'],
                              'n_axis_plunge': mt_properties['n-axis-plunge'],
                              'p_axis_azimuth': mt_properties['p-axis-azimuth'],
                              'p_axis_length' : mt_properties['p-axis-length'],
                              'p_axis_plunge': mt_properties['p-axis-plunge'],
                              't_axis_azimuth': mt_properties['t-axis-azimuth'],
                              't_axis_length' : mt_properties['t-axis-length'],
                              't_axis_plunge': mt_properties['t-axis-plunge'],
                              'percent_double_couple': mt_properties['percent-double-couple'],
                              'scalar_moment': mt_properties['scalar-moment'],
                              'tensor_mpp': mt_properties['tensor-mpp'],
                              'tensor_mrp': mt_properties['tensor-mrp'],
                              'tensor_mrr': mt_properties['tensor-mrr'],
                              'tensor_mrt': mt_properties['tensor-mrt'],
                              'tensor_mtp': mt_properties['tensor-mtp'],
                              'tensor_mtt': mt_properties['tensor-mtt']
                              }
            # print(dict_earthquake)

            # Append the earthquake dictionary to the earthquake list
            earthquake_data_list.append(dict_earthquake)
    except:
        print(f'Error for URL: {url}')

{
    "type": "Feature",
    "properties": {
        "mag": 3.95,
        "place": "13 km SSE of Sierraville, California",
        "time": 1498554561860,
        "updated": 1620398239764,
        "tz": null,
        "url": "https://earthquake.usgs.gov/earthquakes/eventpage/nc72822096",
        "felt": 479,
        "cdi": 4.1,
        "mmi": 3.75,
        "alert": null,
        "status": "reviewed",
        "tsunami": 0,
        "sig": 436,
        "net": "nc",
        "code": "72822096",
        "ids": ",nc72822096,nn00593686,us20009pup,",
        "sources": ",nc,nn,us,",
        "types": ",dyfi,focal-mechanism,moment-tensor,nearby-cities,origin,phase-data,scitech-link,shakemap,",
        "nst": 32,
        "dmin": 0.1018,
        "rms": 0.06,
        "gap": 36,
        "magType": "mw",
        "type": "earthquake",
        "title": "M 4.0 - 13 km SSE of Sierraville, California",
        "products": {
            "dyfi": [
                {
                    "indexid": "18980804",
  

In [192]:
df_earthquake_data = pd.DataFrame(earthquake_data_list)

display(df_earthquake_data)

,id,time,place,longitude,latitude,depth,magnitude,felt,cdi,mmi,...,t_axis_length,t_axis_plunge,percent_double_couple,scalar_moment,tensor_mpp,tensor_mrp,tensor_mrr,tensor_mrt,tensor_mtp,tensor_mtt
0,nc30092964,1995-12-28 18:28:01.230000+00:00,"9 km WNW of Topaz Lake, Nevada",-119.654500,38.714500,-1.011,4.80,NaN,NaN,6.100,...,1.780E+16,8.496,0.94,1.749E+16,1.717E+16,-2.593E+15,-2.563E+14,-1.434E+15,2.590E+15,-1.691E+16
1,nc30092581,1995-12-23 05:39:56.650000+00:00,"8 km WNW of Topaz Lake, Nevada",-119.633000,38.730500,-1.081,4.70,NaN,NaN,NaN,...,1.120E+16,8.968,0.83,1.175E+16,1.082E+16,1.325E+15,-2.496E+14,4.455E+15,1.729E+15,-1.057E+16
2,nc30092506,1995-12-22 09:00:34.560000+00:00,California-Nevada border region,-119.635000,38.721500,3.659,4.86,NaN,NaN,NaN,...,2.764E+16,3.308,0.53,2.435E+16,2.737E+16,-2.897E+15,-1.909E+16,-4.715E+15,-1.774E+15,-8.282E+15
3,nc30091857,1995-12-13 06:25:54.110000+00:00,"9 km ESE of Gilroy, California",-121.470333,36.982167,4.234,3.80,NaN,NaN,NaN,...,7.785E+14,15.899,0.81,7.420E+14,4.317E+14,2.781E+14,-4.776E+13,-3.905E+13,5.545E+14,-3.840E+14
4,nc30094697,1995-12-13 05:45:12.760000+00:00,"9 km ESE of Gilroy, California",-121.470000,36.976667,6.204,3.90,NaN,NaN,NaN,...,8.142E+14,9.016,0.49,9.521E+14,3.951E+14,2.489E+13,2.802E+14,1.330E+14,7.743E+14,-6.753E+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105,nc73834131,2023-01-19 20:30:07.050000+00:00,"16km WSW of Weitchpec, CA",-123.876500,41.128667,26.260,3.66,17.0,3.4,2.318,...,3.729E+14,5.457,0.94,3.783E+14,3.536E+14,-4.656E+13,-2.680E+12,-7.624E+13,-1.057E+14,-3.509E+14
3106,nc73834041,2023-01-19 17:15:55.930000+00:00,"4km S of San Juan Bautista, CA",-121.531000,36.809667,5.320,3.56,218.0,4.3,4.110,...,2.991E+14,2.794,0.68,2.754E+14,2.980E+14,1.766E+13,-5.190E+13,-3.274E+13,1.110E+13,-2.460E+14
3107,nc73830791,2023-01-10 18:32:14.700000+00:00,"6km E of McKinleyville, CA",-124.030333,40.948833,22.200,3.61,726.0,3.6,3.233,...,3.054E+14,3.768,0.76,3.259E+14,2.037E+14,-6.275E+13,-2.460E+14,1.610E+14,1.605E+14,4.230E+13
3108,nc73827571,2023-01-01 18:35:04.510000+00:00,"15km SE of Rio Dell, CA",-123.971000,40.409000,30.630,5.35,976.0,7.2,6.853,...,1.387E+17,10.172,0.79,1.314E+17,1.220E+17,-2.950E+16,-1.183E+16,-9.049E+15,-5.387E+16,-1.102E+17


In [193]:
detail_response = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73827436&format=geojson').json()
print(json.dumps(detail_response, indent=4))

{
    "type": "Feature",
    "properties": {
        "mag": 3.9,
        "place": "9km NW of Pinnacles, CA",
        "time": 1672555750190,
        "updated": 1680711051335,
        "tz": null,
        "url": "https://earthquake.usgs.gov/earthquakes/eventpage/nc73827436",
        "felt": 272,
        "cdi": 4.1,
        "mmi": 4.369,
        "alert": "green",
        "status": "reviewed",
        "tsunami": 0,
        "sig": 346,
        "net": "nc",
        "code": "73827436",
        "ids": ",ew1672555750,nc73827436,us7000j1cv,",
        "sources": ",ew,nc,us,",
        "types": ",dyfi,focal-mechanism,losspager,moment-tensor,nearby-cities,origin,phase-data,scitech-link,shake-alert,shakemap,",
        "nst": 70,
        "dmin": 0.02872,
        "rms": 0.13,
        "gap": 72,
        "magType": "mw",
        "type": "earthquake",
        "title": "M 3.9 - 9km NW of Pinnacles, CA",
        "products": {
            "dyfi": [
                {
                    "indexid": "22555773",
